Chapter 17 Binary Search Trees<br>

Let’s start with an ADT. An ordered mapping is a mapping data type
for which the keys are ordered.

17.1 The Ordered Mapping ADT<br>

An ordered mapping stores a collection of key-value pairs (with comparable
keys) supporting the following operations.
- get(k) - Return the value associate to the key k. An error (KeyError)
is raised if the given key is not present.
- put(k, v) - Add the key-value pair (k,v) to the mapping.
- floor(k) - Return a tuple (k,v) corresponding to the key-value pair
in the mapping with the largest key that is less than or equal to k. If
there is no such tuple, it returns (None, None).
- remove(k) - Remove the key-value pair with key k from the ordered
mapping. An error (KeyError) is raised if the given key is not present.

17.2 Binary Search Tree Properties and Definitions<br>

A tree is called a binary tree if every node has at most two children.<br>
We say that a binary tree is a binary
search tree if for every node x, all the keys in the subtree x.left are less
than the key at x and all the keys in the subtree x.right are greater than
the key of x. This ordering property, also known as the BST property
is what makes a binary search tree different from any other kind of binary
tree.<br>

![Alt text](figs/bst_example1.svg)<br>

The same set of nodes can be arranged into a different binary search
tree.<br>

![Alt text](figs/bst_example2.svg)<br>

The new traversal we introduce here is called inorder
traversal and it visits all the nodes in the left child prior to visiting the
root and then visits all the nodes in the right child after visiting the root.
This order results in a traversal of the nodes in sorted order according to
the ordering of the keys.

17.3 A Minimal implementation

In [10]:
from ds2.mapping import Mapping

class BSTMapping(Mapping):
    def __init__(self):
        self._root = None

    def get(self, key):
        if self._root:
            return self._root.get(key).value
        raise KeyError

    def put(self, key, value):
        if self._root:
            self._root.put(key, value)
        else:
            self._root = BSTNode(key, value)

    def __len__(self):
        return len(self._root) if self._root is not None else 0

    def _entryiter(self):
        if self._root:
            yield from self._root

    def floor(self, key):
        if self._root:
            floornode = self._root.floor(key)
            if floornode is not None:
                return floornode.key, floornode.value
        return None, None

    def remove(self, key):
        if self._root:
            self._root = self._root.remove(key)
        else:
            raise KeyError

    def __delitem__(self, key):
        self.remove(key)

class BSTNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None
        self.length = 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str(self.key) + " : " + str(self.value)

    def get(self, key):
        if key == self.key:
            return key
        elif key < self.key and self.left:
            return self.left.get(key)
        elif key > self.key and self.right:
            return self.right.get(key)
        else:
            raise KeyError

    def put(self, key, value):
        if key == self.key:
            self.value = value
        elif key < self.key:
            if self.left:
                self.left.put(key, value)
            else:
                self.left = BSTNode(key, value)
        elif key > self.key:
            if self.right:
                self.right.put(key, value)
            else:
                self.right = BSTNode(key, value)
        self._updatelength()

    def _updatelength(self):
        len_left = len(self.left) if self.left else 0
        len_right = len(self.right) if self.right else 0
        self._length = 1 + len_left + len_right

    def floor(self, key):
        if key == self.key:
            return self
        elif key < self.key:
            if self.left is not None:
                return self.left.floor(key)
            else:
                return None
        elif key > self.key:
            if self.right is not None:
                floor = self.right.floor(key)
                return floor if floor is not None else self
            else:
                return self
            
    def __iter__(self):
        if self.left is not None:
            yield from self.left
        yield self
        if self.right is not None:
            yield from self.right

    def _swapwith(self, other):
        self.key, other.key = other.key, self.key
        self.value, other.value = other.value, self.value
    
    def maxnode(self):
        return self.right.maxnode() if self.right else self
    
    def remove(self, key):
        if key == self.key:
            if self.left is None: return self.right
            if self.right is None: return self.left
            self._swapwith(self.left.maxnode())
            self.left = self.left.remove(key)
        elif key < self.key and self.left:
            self.left = self.left.remove(key)
        elif key > self.key and self.right:
            self.right = self.right.remove(key)
        else: raise KeyError
        self._updatelength()
        return self

In [11]:
T = BSTMapping()
for i in [3, 2, 1, 6, 4, 5, 9, 8, 10]:
    T[i] = 1
print(T)

{1 : 1, 2 : 1, 3 : 1, 4 : 1, 5 : 1, 6 : 1, 8 : 1, 9 : 1, 10 : 1}


![Alt text](image.png)

In [12]:
T.remove(6)
print(T)

{1 : 1, 2 : 1, 3 : 1, 4 : 1, 5 : 1, 8 : 1, 9 : 1, 10 : 1}


![Alt text](image-1.png)

In [13]:
T.remove(3)
print(T)

{1 : 1, 2 : 1, 4 : 1, 5 : 1, 8 : 1, 9 : 1, 10 : 1}


![Alt text](image-2.png)

This implementation is a top-down approach.<br>

Note: ```yield from self._root``` is the same as ```for item in iter(self._root): yield item```<br>

- put() method:
    - It will work by first doing a binary search in the tree. If it finds the key already in the tree, it overwrites the value (keys in a mapping are unique). Otherwise, when it gets to the bottom of the tree, it adds a new node.
    - The put method also keeps track of the length, i.e. the number of entries in each subtree.

17.3.1 The floor function<br>

The floor function is just a slightly fancier version of get. It also does
a binary search, but it has different behavior when the key is not found,
depending on whether the last search was to the left or to the right. Starting
from any node, if we search to the right and the result is None, then we return
the node itself. If we search to the left and the result is None, we also return
None.<br>

17.3.2 Iteration<br>

As mentioned above, binary search trees support inorder traversal. The
result of an inorder traversal is that the nodes are yielded *in the order of their keys*<br>

17.4 Removal<br>

The start of a removal is to find the node to be removed using binary
search in the tree. Then, if the node is a leaf, we can remove it without
any difficulty. It’s also easy to remove a node with only one child because
we can remove the node and bring its child up without violating the BST
property.<br>

The harder case come when the node to be removed has both a left and
a right child. In that case, we find the node with the largest key in its left
subtree (i.e. the rightmost node). We swap that node with our node to be
removed and call remove again on the left subtree. The next time we reach
that node, we know it will have at most one child, because the node we
swapped it with had no right child<br>